<a href="https://colab.research.google.com/github/mark-ford123/applied_ml_cw_2025_mf/blob/main/coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>